<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>
Author: Yury Kashnitsky, Data Scientist at Mail.Ru Group

This material is subject to the terms and conditions of the license [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Free use is permitted for any non-comercial purpose with an obligatory indication of the names of the authors and of the source.

## <center>Assignment #6. Part 1
### <center> Beating benchmarks in "Catch Me If You Can: Intruder Detection through Webpage Session Tracking"
    
[Competition](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). The task is to beat "Assignment 6 baseline".

In [438]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

Reading original data

In [439]:
PATH_TO_DATA = ('/Users/lucky/.kaggle/competitions/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2')
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id')

Separate target feature 

In [440]:
y = train_df['target']

In [441]:
X_train, X_val, y_train, y_val = train_test_split(train_df, y, test_size=0.2, random_state=42)

Build Tf-Idf features based on sites. You can use `ngram_range`=(1, 3) and `max_features`=100000 or more

In [442]:
%%time

# prepare columns (clean n/a values and convert them to integer then to strings)
site_cols = ['site'+str(i) for i in range(1, 11)]
train_sites = X_train[site_cols].fillna(0).astype(int)
val_sites = X_val[site_cols].fillna(0).astype(int)
test_sites = test_df[site_cols].fillna(0).astype(int)

# TODO: take a number of unique sites into account
train_sites['#unique_sites'] = train_sites.nunique()
val_sites['#unique_sites'] = val_sites.nunique()
test_sites['#unique_sites'] = test_sites.nunique()

train_sessions = train_sites.astype(str).apply(lambda s: ' '.join(s), axis=1)
val_sessions = val_sites.astype(str).apply(lambda s: ' '.join(s), axis=1)
test_sessions = test_sites.astype(str).apply(lambda s: ' '.join(s), axis=1)

# fit TfidfVectorizer with all sites from user sessions
vec = TfidfVectorizer(ngram_range=(1, 3), max_features=100000)
full_sessions = test_sessions.append(train_sessions.append(val_sessions))
vec = vec.fit(full_sessions)

# generate sparse matrices
train_v = vec.transform(train_sessions)
val_v = vec.transform(val_sessions)
test_v = vec.transform(test_sessions)

CPU times: user 1min 2s, sys: 5.14 s, total: 1min 8s
Wall time: 1min 2s


Add features based on the session start time: hour, whether it's morning, day or night and so on.

In [443]:
# separate "timeN" columns into another dataset and clean it up and convert to datetime
timecols = ['time'+str(i) for i in range(1, 11)]
train_times = X_train[timecols].fillna(method='ffill', axis=1).astype('datetime64[ns]')
val_times = X_val[timecols].fillna(method='ffill', axis=1).astype('datetime64[ns]')
test_times = test_df[timecols].fillna(method='ffill', axis=1).astype('datetime64[ns]')

def analyze_time_columns(df):
    df['session_span'] = (df.max(axis=1) - df.min(axis=1)).astype('timedelta64[s]')
    df['session_hour'] = df['time1'].dt.hour
    df['session_pod'] = df['session_hour'].apply(lambda h: 1 if h > 6 and h <= 12 else (
                                                     2 if h > 12 and h <= 18 else (
                                                     3 if h > 18 and h <= 24 
                                                         else 4)))
    df['session_dow'] = df['time1'].dt.dayofweek
    df['is_weekend'] = df['session_dow'].apply(lambda d: 1 if d==5 or d==6 else 0)
    return df[['session_hour', 'session_pod', 'session_dow', 'is_weekend', 'session_span']]
    
time_features_train = analyze_time_columns(train_times)
time_features_val = analyze_time_columns(val_times)
time_features_test = analyze_time_columns(test_times)

Scale this features and combine then with Tf-Idf based on sites (you'll need `scipy.sparse.hstack`)

In [444]:
scaler = StandardScaler()

time_features_train[['session_span']] = scaler.fit_transform(time_features_train[['session_span']])
time_features_val[['session_span']] = scaler.transform(time_features_val[['session_span']])
time_features_test[['session_span']] = scaler.transform(time_features_test[['session_span']])


In [445]:
encoder = OneHotEncoder(categorical_features=[0,1,2,3])
tf_train_mtx = encoder.fit_transform(time_features_train)
tf_val_mtx = encoder.transform(time_features_val)
tf_test_mtx = encoder.transform(time_features_test)


In [446]:
train_dataset = hstack([tf_train_mtx, train_v])
val_dataset = hstack([tf_val_mtx, val_v])
test_dataset = hstack([tf_test_mtx, test_v])

Perform cross-validation with logistic regression.

In [ ]:
%%time
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logitCV = LogisticRegressionCV(
        Cs=10,
        penalty='l2',
        scoring='roc_auc',
        cv=skf,
        random_state=42,
        solver='sag',
        class_weight='balanced',
        max_iter=1000,
        n_jobs=-1
#        tol=10
    )

logitCV.fit(train_dataset, y_train)


os.system('say "your program has finished"')

Let's evaluate our model:

In [448]:
print ('AUC_ROC for our model:', roc_auc_score(y_val, logitCV.predict_proba(val_dataset)[:,1]))

NotFittedError: Call fit before prediction

Experiments:
 * TFxIDF + hour, pod, dow, is_weekend = 0.986696450721
 * TFxIDF + hour, pod, dow, is_weekend, session_span = 0.986712678006

Make prediction for the test set and form a submission file.

In [ ]:
test_pred = logitCV.predict_proba(test_dataset)[:,1]

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)


In [ ]:
write_to_submission_file(test_pred, "assignment6_alice_submission.csv")